In [1]:
import pickle
import pandas as pd

In [2]:
col_names = ['CUI', 'LAT', 'TS', 'LUI', 'STT', 'SUI', 'ISPREF', 'AUI', 'SAUI',
             'SCUI', 'SDUI', 'SAB', 'TTY', 'CODE', 'STR', 'SRL', 'SUPPRESS', 'CVF']

In [3]:
df = pd.read_csv('../2017AA-full/2017AA/META/MRCONSO.RRF', header=None, sep='|')

/opt/mmayers/software/anaconda3/envs/integrate/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (9,10,13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
print(len(df))
print(df[18].count())

13518565
0


In [5]:
# Fix the fact that rows end with a pipe, producing an extra entirely empty column
df = df.iloc[:, :-1]

In [6]:
df.columns = col_names

In [7]:
df.head(5)

,CUI,LAT,TS,LUI,STT,SUI,ISPREF,AUI,SAUI,SCUI,SDUI,SAB,TTY,CODE,STR,SRL,SUPPRESS,CVF
0,C0000005,ENG,P,L0000005,PF,S0007492,Y,A26634265,NaN,M0019694,D012711,MSH,PEP,D012711,(131)I-Macroaggregated Albumin,0,N,256.0
1,C0000005,ENG,S,L0270109,PF,S0007491,Y,A26634266,NaN,M0019694,D012711,MSH,ET,D012711,(131)I-MAA,0,N,256.0
2,C0000005,FRE,P,L6220710,PF,S7133957,Y,A13433185,NaN,M0019694,D012711,MSHFRE,PEP,D012711,Macroagrégats d'albumine marquée à l'iode 131,3,N,NaN
3,C0000005,FRE,S,L6215648,PF,S7133916,Y,A27488794,NaN,M0019694,D012711,MSHFRE,ET,D012711,MAA-I 131,3,N,NaN
4,C0000005,FRE,S,L6215656,PF,S7133956,Y,A27614225,NaN,M0019694,D012711,MSHFRE,ET,D012711,Macroagrégats d'albumine humaine marquée à l'i...,3,N,NaN


In [8]:
msh_rows = df.query('SAB == "MSH"')

In [9]:
grpd = msh_rows.groupby('CUI')
cui_to_msh = dict()
for label, df in grpd:
    cui_to_msh[label] = list(set(df['SDUI']))
    

In [10]:
print(len(cui_to_msh))

366386


In [11]:
# There are a few with multiple mappings...
for v in cui_to_msh.values():
    if len(v) > 1:
        print(v)

['C000613045', 'C469892']
['C565753', 'C536684']
['C000612721', 'D013390']
['C540133', 'C000600295']
['C000613846', 'C052620']
['C008976', 'C000595128']
['C423967', 'C000610146']
['D011070', 'C000609731']
['C014096', 'C000611472']
['C536345', 'D005413']
['C537539', 'C537536']
['D002707', 'C586041']
['C416903', 'D000519']
['C000609658', 'C037508']
['C538162', 'C563821']
['C503999', 'D027381']
['C000605502', 'D000071677']
['C086742', 'C043567']
['D008751', 'C469427']
['D020434', 'C564661']
['C057224', 'C449219']
['C038678', 'C063153']
['C006753', 'C014081']
['C000606873', 'C540264']
['C574884', 'C004315']
['C000605855', 'C000613559']
['C032538', 'C042041']
['C000605449', 'C571874']
['C000605628', 'C574187']
['C114026', 'C093788']
['C026055', 'C400713']
['C000606932', 'C040008']
['C099565', 'C475721']
['C035125', 'C000611492']
['C011394', 'C456820']
['C000611407', 'C444299']
['C586680', 'C000609785']
['C008430', 'C479199']
['C486292', 'C000613139']
['C410423', 'C000609666']
['C090722', 'C

In [12]:
pickle.dump(cui_to_msh, open('../data/UMLS-CUI_to_MeSH-Descripctor.pkl', 'wb'))

## Get a quick and dirty name map for the 1to1 id to values

In [13]:
cui_to_msh_1t1 = {k: v[0] for k, v in cui_to_msh.items() if len(v) == 1}

Just take any mapping... we'll get more "Correct" names for as many as we can later

In [14]:
cuis = list(cui_to_msh_1t1.keys())
to_name = msh_rows.query('CUI in @cuis')
msh_to_name = to_name.drop_duplicates(subset=["SDUI"]).set_index('SDUI')['STR'].to_dict()

In [15]:
pickle.dump(msh_to_name, open('../data/MeSH_to_name_quick_n_dirty.pkl', 'wb'))